## Summary
This project involves implementing the Sessa Empirical Estimator (SEE) for estimating prescription durations in a pharmacoepidemiological context. Originally written in R, the SEE will be ported to Python, and applied to a chosen dataset. The standard SEE approach uses K-Means clustering to group similar prescription intervals, but this project will also explore an alternative clustering algorithm to see if it produces different insights. Finally, the results and performance from both clustering methods will be compared, highlighting any differences in exposure classification or prescription duration estimates.